# TV Script Generation

In this project, you'll generate your own [Seinfeld](https://en.wikipedia.org/wiki/Seinfeld) TV scripts using RNNs.  You'll be using part of the [Seinfeld dataset](https://www.kaggle.com/thec03u5/seinfeld-chronicles#scripts.csv) of scripts from 9 seasons.  The Neural Network you'll build will generate a new ,"fake" TV script, based on patterns it recognizes in this training data.

## Get the Data

The data is already provided for you in `./data/Seinfeld_Scripts.txt` and you're encouraged to open that file and look at the text. 
>* As a first step, we'll load in this data and look at some samples. 
* Then, you'll be tasked with defining and training an RNN to generate a new script!

In [1]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
# load in data
import helper
data_dir = './data/Seinfeld_Scripts.txt'
text = helper.load_data(data_dir)

## Explore the Data
Play around with `view_line_range` to view different parts of the data. This will give you a sense of the data you'll be working with. You can see, for example, that it is all lowercase text, and each new line of dialogue is separated by a newline character `\n`.

In [2]:
view_line_range = (0, 10)

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
import numpy as np

print('Dataset Stats')
print('Roughly the number of unique words: {}'.format(len({word: None for word in text.split()})))

lines = text.split('\n')
print('Number of lines: {}'.format(len(lines)))
word_count_line = [len(line.split()) for line in lines]
print('Average number of words in each line: {}'.format(np.average(word_count_line)))

print('The lines {} to {}:'.format(*view_line_range))
print('\n'.join(text.split('\n')[view_line_range[0]:view_line_range[1]]))

Dataset Stats
Roughly the number of unique words: 46367
Number of lines: 109233
Average number of words in each line: 5.544240293684143
The lines 0 to 10:
jerry: do you know what this is all about? do you know, why were here? to be out, this is out...and out is one of the single most enjoyable experiences of life. people...did you ever hear people talking about we should go out? this is what theyre talking about...this whole thing, were all out now, no one is home. not one person here is home, were all out! there are people trying to find us, they dont know where we are. (on an imaginary phone) did you ring?, i cant find him. where did he go? he didnt tell me where he was going. he must have gone out. you wanna go out you get ready, you pick out the clothes, right? you take the shower, you get all ready, get the cash, get your friends, the car, the spot, the reservation...then youre standing around, what do you do? you go we gotta be getting back. once youre out, you wanna get back! yo

---
## Implement Pre-processing Functions
The first thing to do to any dataset is pre-processing.  Implement the following pre-processing functions below:
- Lookup Table
- Tokenize Punctuation

### Lookup Table
To create a word embedding, you first need to transform the words to ids.  In this function, create two dictionaries:
- Dictionary to go from the words to an id, we'll call `vocab_to_int`
- Dictionary to go from the id to word, we'll call `int_to_vocab`

Return these dictionaries in the following **tuple** `(vocab_to_int, int_to_vocab)`

In [3]:
import problem_unittests as tests
import collections

def create_lookup_tables(text):
    """
    Create lookup tables for vocabulary
    :param text: The text of tv scripts split into words
    :return: A tuple of dicts (vocab_to_int, int_to_vocab)
    """
    # Count and sort the corpus
    word_counts = collections.Counter(text)
    sorted_counts = word_counts.most_common()
    
    # create the look up dictionaries
    int_to_vocab = {n: word_tuple[0] for n, word_tuple in enumerate(sorted_counts)}
    vocab_to_int = {word: n for n, word in int_to_vocab.items()}

    return (vocab_to_int, int_to_vocab)


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_create_lookup_tables(create_lookup_tables)

Tests Passed


### Tokenize Punctuation
We'll be splitting the script into a word array using spaces as delimiters.  However, punctuations like periods and exclamation marks can create multiple ids for the same word. For example, "bye" and "bye!" would generate two different word ids.

Implement the function `token_lookup` to return a dict that will be used to tokenize symbols like "!" into "||Exclamation_Mark||".  Create a dictionary for the following symbols where the symbol is the key and value is the token:
- Period ( **.** )
- Comma ( **,** )
- Quotation Mark ( **"** )
- Semicolon ( **;** )
- Exclamation mark ( **!** )
- Question mark ( **?** )
- Left Parentheses ( **(** )
- Right Parentheses ( **)** )
- Dash ( **-** )
- Return ( **\n** )

This dictionary will be used to tokenize the symbols and add the delimiter (space) around it.  This separates each symbols as its own word, making it easier for the neural network to predict the next word. Make sure you don't use a value that could be confused as a word; for example, instead of using the value "dash", try using something like "||dash||".

In [4]:
def token_lookup():
    """
    Generate a dict to turn punctuation into a token.
    :return: Tokenized dictionary where the key is the punctuation and the value is the token
    """
    return {
        '.': '||period||',
        ',': '||come||',
        '"': '||doublequote||',
        ';': '||semicolon||',
        '!': '||exclamation||',
        '?': '||questionmark||',
        '(': '||lparenth||',
        ')': '||rparenth||',
        '-': '||dash||',
        '\n': '||newline||'
    }

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_tokenize(token_lookup)

Tests Passed


## Pre-process all the data and save it

Running the code cell below will pre-process all the data and save it to file. You're encouraged to look at the code for `preprocess_and_save_data` in the `helpers.py` file to see what it's doing in detail, but you do not need to change this code.

In [5]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
# pre-process training data
helper.preprocess_and_save_data(data_dir, token_lookup, create_lookup_tables)

# Check Point
This is your first checkpoint. If you ever decide to come back to this notebook or have to restart the notebook, you can start from here. The preprocessed data has been saved to disk.

In [6]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import torch
import numpy as np
import torch.nn as nn
import helper
import problem_unittests as tests
from tqdm import tqdm

int_text, vocab_to_int, int_to_vocab, token_dict = helper.load_preprocess()

## Build the Neural Network
In this section, you'll build the components necessary to build an RNN by implementing the RNN Module and forward and backpropagation functions.

### Check Access to GPU

In [26]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import torch

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# Check for a GPU
train_on_gpu = torch.cuda.is_available()
if not train_on_gpu:
    print('No GPU found. Please use a GPU to train your neural network.')
    

## Input
Let's start with the preprocessed input data. We'll use [TensorDataset](http://pytorch.org/docs/master/data.html#torch.utils.data.TensorDataset) to provide a known format to our dataset; in combination with [DataLoader](http://pytorch.org/docs/master/data.html#torch.utils.data.DataLoader), it will handle batching, shuffling, and other dataset iteration functions.

You can create data with TensorDataset by passing in feature and target tensors. Then create a DataLoader as usual.
```
data = TensorDataset(feature_tensors, target_tensors)
data_loader = torch.utils.data.DataLoader(data, 
                                          batch_size=batch_size)
```

### Batching
Implement the `batch_data` function to batch `words` data into chunks of size `batch_size` using the `TensorDataset` and `DataLoader` classes.

>You can batch words using the DataLoader, but it will be up to you to create `feature_tensors` and `target_tensors` of the correct size and content for a given `sequence_length`.

For example, say we have these as input:
```
words = [1, 2, 3, 4, 5, 6, 7]
sequence_length = 4
```

Your first `feature_tensor` should contain the values:
```
[1, 2, 3, 4]
```
And the corresponding `target_tensor` should just be the next "word"/tokenized word value:
```
5
```
This should continue with the second `feature_tensor`, `target_tensor` being:
```
[2, 3, 4, 5]  # features
6             # target
```

In [8]:
from torch.utils.data import TensorDataset, DataLoader

def batch_data(words, sequence_length, batch_size):
    """
    Batch the neural network data using DataLoader
    :param words: The word ids of the TV scripts
    :param sequence_length: The sequence length of each batch
    :param batch_size: The size of each batch; the number of sequences in a batch
    :return: DataLoader with batched data
    """    
    # get number of targets we can make (must create full sequences)
    n_targets = len(words) - sequence_length

    # create the targets and features
    features, targets = [], []
    for i in range(n_targets):
        features.append(words[i : i+sequence_length])
        targets.append(words[i+sequence_length])
    
    # convert Python list to PyTroch Tensors
    features, targets = np.asarray(features), np.asarray(targets)
    features, targets = torch.from_numpy(features), torch.from_numpy(targets)
    
    # instanciate PyTorch's dataset class and DataLoader
    dataset = TensorDataset(features, targets)
    dataloader = DataLoader(dataset, shuffle=True, batch_size=batch_size)
    
    return dataloader

### Test your dataloader 

You'll have to modify this code to test a batching function, but it should look fairly similar.

Below, we're generating some test text data and defining a dataloader using the function you defined, above. Then, we are getting some sample batch of inputs `sample_x` and targets `sample_y` from our dataloader.

Your code should return something like the following (likely in a different order, if you shuffled your data):

```
torch.Size([10, 5])
tensor([[ 28,  29,  30,  31,  32],
        [ 21,  22,  23,  24,  25],
        [ 17,  18,  19,  20,  21],
        [ 34,  35,  36,  37,  38],
        [ 11,  12,  13,  14,  15],
        [ 23,  24,  25,  26,  27],
        [  6,   7,   8,   9,  10],
        [ 38,  39,  40,  41,  42],
        [ 25,  26,  27,  28,  29],
        [  7,   8,   9,  10,  11]])

torch.Size([10])
tensor([ 33,  26,  22,  39,  16,  28,  11,  43,  30,  12])
```

### Sizes
Your sample_x should be of size `(batch_size, sequence_length)` or (10, 5) in this case and sample_y should just have one dimension: batch_size (10). 

### Values

You should also notice that the targets, sample_y, are the *next* value in the ordered test_text data. So, for an input sequence `[ 28,  29,  30,  31,  32]` that ends with the value `32`, the corresponding output should be `33`.

In [9]:
# test dataloader

test_text = range(50)
t_loader = batch_data(test_text, sequence_length=5, batch_size=10)

data_iter = iter(t_loader)
sample_x, sample_y = data_iter.next()

print(sample_x.shape)
print(sample_x)
print()
print(sample_y.shape)
print(sample_y)

torch.Size([10, 5])
tensor([[41, 42, 43, 44, 45],
        [43, 44, 45, 46, 47],
        [ 8,  9, 10, 11, 12],
        [27, 28, 29, 30, 31],
        [38, 39, 40, 41, 42],
        [ 5,  6,  7,  8,  9],
        [ 3,  4,  5,  6,  7],
        [37, 38, 39, 40, 41],
        [29, 30, 31, 32, 33],
        [20, 21, 22, 23, 24]], dtype=torch.int32)

torch.Size([10])
tensor([46, 48, 13, 32, 43, 10,  8, 42, 34, 25], dtype=torch.int32)


---
## Build the Neural Network
Implement an RNN using PyTorch's [Module class](http://pytorch.org/docs/master/nn.html#torch.nn.Module). You may choose to use a GRU or an LSTM. To complete the RNN, you'll have to implement the following functions for the class:
 - `__init__` - The initialize function. 
 - `init_hidden` - The initialization function for an LSTM/GRU hidden state
 - `forward` - Forward propagation function.
 
The initialize function should create the layers of the neural network and save them to the class. The forward propagation function will use these layers to run forward propagation and generate an output and a hidden state.

**The output of this model should be the *last* batch of word scores** after a complete sequence has been processed. That is, for each input sequence of words, we only want to output the word scores for a single, most likely, next word.

### Hints

1. Make sure to stack the outputs of the lstm to pass to your fully-connected layer, you can do this with `lstm_output = lstm_output.contiguous().view(-1, self.hidden_dim)`
2. You can get the last batch of word scores by shaping the output of the final, fully-connected layer like so:

```
# reshape into (batch_size, seq_length, output_size)
output = output.view(batch_size, -1, self.output_size)
# get last batch
out = output[:, -1]
```

In [55]:
import torch.nn as nn
import torch.nn.functional as functional

class RNN(nn.Module):
    
    def __init__(self, vocab_size, output_size, embedding_dim, hidden_dim, n_layers, dropout=0.5):
        """
        Initialize the PyTorch RNN Module
        :param vocab_size: The number of input dimensions of the neural network (the size of the vocabulary)
        :param output_size: The number of output dimensions of the neural network
        :param embedding_dim: The size of embeddings, should you choose to use them        
        :param hidden_dim: The size of the hidden layer outputs
        :param dropout: dropout to add in between LSTM/GRU layers
        """
        super(RNN, self).__init__()
        
        # init hidden weights params
        self.n_layers = n_layers
        self.hidden_dim = hidden_dim
        self.vocab_size = vocab_size

        # define the embedding layer
        self.embedding = nn.Embedding(vocab_size, embedding_dim)

        # define the LSTM layer
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, n_layers, 
                            dropout=dropout, batch_first=True)

        # define fully-connected layer
        self.dense = nn.Linear(hidden_dim, output_size)
    
    
    def forward(self, nn_input, hidden):
        """
        Forward propagation of the neural network
        :param nn_input: The input to the neural network
        :param hidden: The hidden state        
        :return: Two Tensors, the output of the neural network and the latest hidden state
        """    
        # ensure embedding layer gets a LongTensor input
        nn_input = nn_input.long()
        
        # get the batch size for reshaping
        batch_size = nn_input.size(0)
        
        ## define forward pass
        embed = self.embedding(nn_input)
        output, state = self.lstm(embed, hidden)
        
        # stack LSTM
        output = output.contiguous().view(-1, self.hidden_dim) 
        
        # pass through last fully connected layer
        output = self.dense(output)
        
        output = output.view(batch_size, -1, self.vocab_size)
        output = output[:, -1] # save only the last output
        
        # return one batch of output word scores and the hidden state
        return output, state   

    
    def init_hidden(self, batch_size):
        '''
        Initialize the hidden state of an LSTM
        :param batch_size: The batch_size of the hidden state
        :return: hidden state of dims (n_layers, batch_size, hidden_dim)
        '''
        # Create two new tensors with sizes n_layers x batch_size x n_hidden,
        # initialized to zero, for hidden state and cell state of LSTM
        weight = next(self.parameters()).data
        
        if (torch.cuda.is_available()): #
            hidden = (weight.new(self.n_layers, batch_size, self.hidden_dim).zero_().cuda(),
                      weight.new(self.n_layers, batch_size, self.hidden_dim).zero_().cuda())
        else:
            hidden = (weight.new(self.n_layers, batch_size, self.hidden_dim).zero_(),
                      weight.new(self.n_layers, batch_size, self.hidden_dim).zero_())
        
        return hidden
        
        
"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_rnn(RNN, train_on_gpu)

Tests Passed


### Define forward and backpropagation

Use the RNN class you implemented to apply forward and back propagation. This function will be called, iteratively, in the training loop as follows:
```
loss = forward_back_prop(decoder, decoder_optimizer, criterion, inp, target)
```

And it should return the average loss over a batch and the hidden state returned by a call to `RNN(inp, hidden)`. Recall that you can get this loss by computing it, as usual, and calling `loss.item()`.

**If a GPU is available, you should move your data to that GPU device, here.**

In [11]:
def forward_back_prop(rnn, optimizer, criterion, inp, target, hidden):
    """
    Forward and backward propagation on the neural network
    :param rnn: The PyTorch Module that holds the neural network
    :param optimizer: The PyTorch optimizer for the neural network
    :param criterion: The PyTorch loss function
    :param inp: A batch of input to the neural network
    :param target: The target output for the batch of input
    :return: The loss and the latest hidden state Tensor
    """
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    
    # move data to GPU, if available
    rnn.to(device)
    inp, target = inp.to(device), target.to(device)
    
    # dismember the hidden states to prevent backprop through entire training history
    hidden = tuple([hid.data for hid in hidden])
    
    # zero accumulated gradients
    rnn.zero_grad()

    # get the output and hidden state from the model
    output, hidden = rnn(inp, hidden)
    
    # calcualte the loss
    loss = criterion(output.squeeze(), target.long())
    
    # perform backpropagation
    loss.backward()
    
    # clip to prevent gradients from becoming too large before optimizating
    # nn.utils.clip_grad_norm_(rnn.parameters(), 4) # CLIPS POST OPTIMIZING ACCORDING TO THE DOCS
    nn.utils.clip_grad_value_(rnn.parameters(), 4)
    optimizer.step()
    
    # ensure everything is sent back to cpu processing
    rnn.to('cpu')
    inp, target = inp.to('cpu'), target.to('cpu')
    
    # return the loss over a batch and the hidden state produced by our model
    return loss.item(), hidden

# Note that these tests aren't completely extensive.
# they are here to act as general checks on the expected outputs of your functions
"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_forward_back_prop(RNN, forward_back_prop, train_on_gpu)

Tests Passed


## Neural Network Training

With the structure of the network complete and data ready to be fed in the neural network, it's time to train it.

### Train Loop

The training loop is implemented for you in the `train_decoder` function. This function will train the network over all the batches for the number of epochs given. The model progress will be shown every number of batches. This number is set with the `show_every_n_batches` parameter. You'll set this parameter along with other parameters in the next section.

In [12]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""

def train_rnn(rnn, batch_size, optimizer, criterion, n_epochs, show_every_n_batches=100):
    batch_losses = []
    
    rnn.train()

    print("Training for %d epoch(s)..." % n_epochs)
    for epoch_i in tqdm(range(1, n_epochs + 1)):
        
        # initialize hidden state
        hidden = rnn.init_hidden(batch_size)
        
        for batch_i, (inputs, labels) in enumerate(train_loader, 1):
            
            # make sure you iterate over completely full batches, only
            n_batches = len(train_loader.dataset)//batch_size
            if(batch_i > n_batches):
                break
            
            # forward, back prop
            loss, hidden = forward_back_prop(rnn, optimizer, criterion, inputs, labels, hidden)          
            # record loss
            batch_losses.append(loss)

            # printing loss stats
            if batch_i % show_every_n_batches == 0:
                print('Epoch: {:>4}/{:<4}  Loss: {}\n'.format(
                    epoch_i, n_epochs, np.average(batch_losses)))
                batch_losses = []

    # returns a trained rnn
    return rnn

### Hyperparameters

Set and train the neural network with the following parameters:
- Set `sequence_length` to the length of a sequence.
- Set `batch_size` to the batch size.
- Set `num_epochs` to the number of epochs to train for.
- Set `learning_rate` to the learning rate for an Adam optimizer.
- Set `vocab_size` to the number of unique tokens in our vocabulary.
- Set `output_size` to the desired size of the output.
- Set `embedding_dim` to the embedding dimension; smaller than the vocab_size.
- Set `hidden_dim` to the hidden dimension of your RNN.
- Set `n_layers` to the number of layers/cells in your RNN.
- Set `show_every_n_batches` to the number of batches at which the neural network should print progress.

If the network isn't getting the desired results, tweak these parameters and/or the layers in the `RNN` class.

In [13]:
# Data params
# Sequence Length
sequence_length = 6 #13  # of words in a sequence
# Batch Size
batch_size = 32 # 32

# data loader - do not change
train_loader = batch_data(int_text, sequence_length, batch_size)

In [14]:
# Training parameters
# Number of Epochs
num_epochs = 30 #
# Learning Rate
learning_rate = 0.0005 # 0.001 0.002, 0.0015

# Model parameters
# Vocab size
vocab_size = len(int_to_vocab)
# Output size
output_size = vocab_size
# Embedding Dimension
embedding_dim = 400 # 300
# Hidden Dimension
hidden_dim = 512
# Number of RNN Layers
n_layers = 2 

# Show stats for every n number of batches
show_every_n_batches = 500

### Train
In the next cell, you'll train the neural network on the pre-processed data.  If you have a hard time getting a good loss, you may consider changing your hyperparameters. In general, you may get better results with larger hidden and n_layer dimensions, but larger models take a longer time to train. 
> **You should aim for a loss less than 3.5.** 

You should also experiment with different sequence lengths, which determine the size of the long range dependencies that a model can learn.

In [15]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""

# create model and move to gpu if available
rnn = RNN(vocab_size, output_size, embedding_dim, hidden_dim, n_layers, dropout=0.5)
if train_on_gpu:
    rnn.cuda()

# defining loss and optimization functions for training
optimizer = torch.optim.Adam(rnn.parameters(), lr=learning_rate)
criterion = nn.CrossEntropyLoss()

# training the model
trained_rnn = train_rnn(rnn, batch_size, optimizer, criterion, num_epochs, show_every_n_batches)

# saving the trained model
helper.save_model('./save/trained_rnn', trained_rnn)
print('Model Trained and Saved')

  0%|                                                                                           | 0/30 [00:00<?, ?it/s]

Training for 30 epoch(s)...
Epoch:    1/30    Loss: 5.631917074203491

Epoch:    1/30    Loss: 5.097865256309509

Epoch:    1/30    Loss: 4.950217853069305

Epoch:    1/30    Loss: 4.823883620262146

Epoch:    1/30    Loss: 4.684354959011078

Epoch:    1/30    Loss: 4.631889041423798

Epoch:    1/30    Loss: 4.6176121168136595

Epoch:    1/30    Loss: 4.518433081150055

Epoch:    1/30    Loss: 4.498312164783478

Epoch:    1/30    Loss: 4.472153312683106

Epoch:    1/30    Loss: 4.455683742523194

Epoch:    1/30    Loss: 4.381301685333252

Epoch:    1/30    Loss: 4.4505188164711

Epoch:    1/30    Loss: 4.368369184017181

Epoch:    1/30    Loss: 4.34871819114685

Epoch:    1/30    Loss: 4.325979486942291

Epoch:    1/30    Loss: 4.319601820468902

Epoch:    1/30    Loss: 4.282609702110291

Epoch:    1/30    Loss: 4.309314945697785

Epoch:    1/30    Loss: 4.273182336807251

Epoch:    1/30    Loss: 4.276075323581695

Epoch:    1/30    Loss: 4.300496603488922

Epoch:    1/30    Loss: 4.23

  3%|██▌                                                                         | 1/30 [1:28:47<42:54:59, 5327.58s/it]

Epoch:    2/30    Loss: 4.008505147247488

Epoch:    2/30    Loss: 3.918963326931

Epoch:    2/30    Loss: 3.8957852029800417

Epoch:    2/30    Loss: 3.9429426741600038

Epoch:    2/30    Loss: 3.938434164047241

Epoch:    2/30    Loss: 3.9215558671951296

Epoch:    2/30    Loss: 3.9544380650520323

Epoch:    2/30    Loss: 3.9503340520858763

Epoch:    2/30    Loss: 3.9450570974349977

Epoch:    2/30    Loss: 3.938774064064026

Epoch:    2/30    Loss: 3.956159104347229

Epoch:    2/30    Loss: 3.926280095100403

Epoch:    2/30    Loss: 3.9352136788368224

Epoch:    2/30    Loss: 3.9956078724861146

Epoch:    2/30    Loss: 3.924984833240509

Epoch:    2/30    Loss: 3.968446300983429

Epoch:    2/30    Loss: 4.028602971076966

Epoch:    2/30    Loss: 3.9883325910568237

Epoch:    2/30    Loss: 3.990843740463257

Epoch:    2/30    Loss: 3.9720629196166994

Epoch:    2/30    Loss: 4.01558452796936

Epoch:    2/30    Loss: 3.9775825123786928

Epoch:    2/30    Loss: 3.9807479090690614

Epo

  7%|█████                                                                       | 2/30 [2:57:09<41:19:04, 5312.32s/it]

...

Epoch:   29/30    Loss: 3.358306271883241

Epoch:   29/30    Loss: 3.238641005754471

Epoch:   29/30    Loss: 3.27620326089859

Epoch:   29/30    Loss: 3.285249485731125

Epoch:   29/30    Loss: 3.2541409142017366

Epoch:   29/30    Loss: 3.2915873465538024

Epoch:   29/30    Loss: 3.2716327471733093

Epoch:   29/30    Loss: 3.3077684829235077

Epoch:   29/30    Loss: 3.3378350074291228

Epoch:   29/30    Loss: 3.281961927652359

Epoch:   29/30    Loss: 3.3055741658210755

Epoch:   29/30    Loss: 3.302669280529022

Epoch:   29/30    Loss: 3.3171240990161897

Epoch:   29/30    Loss: 3.3464309134483337

Epoch:   29/30    Loss: 3.3442230968475344

Epoch:   29/30    Loss: 3.338314121723175

Epoch:   29/30    Loss: 3.3245060505867006

Epoch:   29/30    Loss: 3.349236986160278

Epoch:   29/30    Loss: 3.371635474205017

Epoch:   29/30    Loss: 3.3510051505565643

Epoch:   29/30    Loss: 3.403738645553589

Epoch:   29/30    Loss: 3.3795646941661834

Epoch:   29/30    Loss: 3.354976163625717



 97%|████████████████████████████████████████████████████████████████████████▌  | 29/30 [43:07:44<1:29:29, 5369.24s/it]

Epoch:   30/30    Loss: 3.3507579282638966

Epoch:   30/30    Loss: 3.2819102745056155

Epoch:   30/30    Loss: 3.3065697944164274

Epoch:   30/30    Loss: 3.268867141723633

Epoch:   30/30    Loss: 3.280075402498245

Epoch:   30/30    Loss: 3.2668852715492247

Epoch:   30/30    Loss: 3.287214071273804

Epoch:   30/30    Loss: 3.2565506172180174

Epoch:   30/30    Loss: 3.280714129924774

Epoch:   30/30    Loss: 3.3060995745658874

Epoch:   30/30    Loss: 3.349662329673767

Epoch:   30/30    Loss: 3.3313460311889647

Epoch:   30/30    Loss: 3.286267727136612

Epoch:   30/30    Loss: 3.315082930326462

Epoch:   30/30    Loss: 3.319010198354721

Epoch:   30/30    Loss: 3.3348175873756407

Epoch:   30/30    Loss: 3.3184071786403657

Epoch:   30/30    Loss: 3.276852245807648

Epoch:   30/30    Loss: 3.3334936633110046

Epoch:   30/30    Loss: 3.3593792395591735

Epoch:   30/30    Loss: 3.3554242124557496

Epoch:   30/30    Loss: 3.3976867225170135

Epoch:   30/30    Loss: 3.373105731725692

100%|█████████████████████████████████████████████████████████████████████████████| 30/30 [44:36:52<00:00, 5353.75s/it]


Model Trained and Saved


In [24]:
# continue training :(

# save states 
state = {'epoch': num_epochs + 1, 'state_dict': trained_rnn.state_dict(),
             'optimizer': optimizer.state_dict()}

filename = 'trained30_rnn.pt'
torch.save(state, filename)

model = rnn = RNN(vocab_size, output_size, embedding_dim, hidden_dim, n_layers, dropout=0.5)
opt = torch.optim.Adam(rnn.parameters(), lr=learning_rate)

def load_checkpoint(model, optimizer, filename):
    '''
    Note: Input model & optimizer should be pre-defined.  This routine only updates their states.
    '''
    start_epoch = 0
    checkpoint = torch.load(filename)
    start_epoch = checkpoint['epoch']
    model.load_state_dict(checkpoint['state_dict'])
    optimizer.load_state_dict(checkpoint['optimizer'])
    
    return model, optimizer, start_epoch

model, opt, start_epoch = load_checkpoint(model, optimizer, filename=filename)

In [28]:
device = 'cpu' # avoiding device erros
model = model.to(device)
# now individually transfer the optimizer parts...
for state in opt.state.values():
    for k, v in state.items():
        if isinstance(v, torch.Tensor):
            state[k] = v.to(device)
            
# training the model
model = train_rnn(model, batch_size, opt, criterion, 2, show_every_n_batches)

  0%|                                                                                            | 0/2 [00:00<?, ?it/s]

Training for 2 epoch(s)...
Epoch:    1/2     Loss: 3.230847548723221

Epoch:    1/2     Loss: 3.242536901950836

Epoch:    1/2     Loss: 3.226005310535431

Epoch:    1/2     Loss: 3.284716125488281

Epoch:    1/2     Loss: 3.238082037448883

Epoch:    1/2     Loss: 3.2779522089958193

Epoch:    1/2     Loss: 3.2803900933265684

Epoch:    1/2     Loss: 3.2567694537639618

Epoch:    1/2     Loss: 3.230153357028961

Epoch:    1/2     Loss: 3.2203620710372927

Epoch:    1/2     Loss: 3.2418822095394133

Epoch:    1/2     Loss: 3.246777020454407

Epoch:    1/2     Loss: 3.2307465674877167

Epoch:    1/2     Loss: 3.2492629382610323

Epoch:    1/2     Loss: 3.2557648549079894

Epoch:    1/2     Loss: 3.2403375010490416

Epoch:    1/2     Loss: 3.220504664182663

Epoch:    1/2     Loss: 3.2442013556957243

Epoch:    1/2     Loss: 3.238982215166092

Epoch:    1/2     Loss: 3.236511000394821

Epoch:    1/2     Loss: 3.2021658787727354

Epoch:    1/2     Loss: 3.266299452781677

Epoch:    1/2   

 50%|██████████████████████████████████████▌                                      | 1/2 [3:28:03<3:28:03, 12483.05s/it]

Epoch:    2/2     Loss: 3.235151591224931

Epoch:    2/2     Loss: 3.233255350112915

Epoch:    2/2     Loss: 3.2245744981765747

Epoch:    2/2     Loss: 3.2616281716823576

Epoch:    2/2     Loss: 3.213267842292786

Epoch:    2/2     Loss: 3.20316557598114

Epoch:    2/2     Loss: 3.2736718525886537

Epoch:    2/2     Loss: 3.2411373143196105

Epoch:    2/2     Loss: 3.25697576880455

Epoch:    2/2     Loss: 3.246842366218567

Epoch:    2/2     Loss: 3.2621957981586456

Epoch:    2/2     Loss: 3.1940839657783506

Epoch:    2/2     Loss: 3.2663290660381317

Epoch:    2/2     Loss: 3.2394476187229158

Epoch:    2/2     Loss: 3.2552397892475127

Epoch:    2/2     Loss: 3.265643047809601

Epoch:    2/2     Loss: 3.268998483419418

Epoch:    2/2     Loss: 3.219040473937988

Epoch:    2/2     Loss: 3.2335102310180663

Epoch:    2/2     Loss: 3.236566183805466

Epoch:    2/2     Loss: 3.257062754154205

Epoch:    2/2     Loss: 3.288965368747711

Epoch:    2/2     Loss: 3.235949367046356

Epo

100%|███████████████████████████████████████████████████████████████████████████████| 2/2 [7:01:05<00:00, 12632.90s/it]


In [45]:
state = {'epoch': 33, 'state_dict': model.state_dict(), 'optimizer': opt.state_dict()}

filename = 'trained_rnn32.pt'
torch.save(state, filename)
helper.save_model('./save/trained_rnn', model)

### Question: How did you decide on your model hyperparameters? 
For example, did you try different sequence_lengths and find that one size made the model converge faster? What about your hidden_dim and n_layers; how did you decide on those?

**Answer:** (Write answer, here)

---
# Checkpoint

After running the above training cell, your model will be saved by name, `trained_rnn`, and if you save your notebook progress, **you can pause here and come back to this code at another time**. You can resume your progress by running the next cell, which will load in our word:id dictionaries _and_ load in your saved model by name!

In [52]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import torch
import helper
import problem_unittests as tests

_, vocab_to_int, int_to_vocab, token_dict = helper.load_preprocess()
trained_rnn = helper.load_model('./save/trained_rnn')

## Generate TV Script
With the network trained and saved, you'll use it to generate a new, "fake" Seinfeld TV script in this section.

### Generate Text
To generate the text, the network needs to start with a single word and repeat its predictions until it reaches a set length. You'll be using the `generate` function to do this. It takes a word id to start with, `prime_id`, and generates a set length of text, `predict_len`. Also note that it uses topk sampling to introduce some randomness in choosing the most likely next word, given an output set of word scores!

In [69]:
"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
import torch.nn.functional as F

def generate(rnn, prime_id, int_to_vocab, token_dict, pad_value, predict_len=100):
    """
    Generate text using the neural network
    :param decoder: The PyTorch Module that holds the trained neural network
    :param prime_id: The word id to start the first prediction
    :param int_to_vocab: Dict of word id keys to word values
    :param token_dict: Dict of puncuation tokens keys to puncuation values
    :param pad_value: The value used to pad a sequence
    :param predict_len: The length of text to generate
    :return: The generated text
    """
    rnn.eval()
    
    # create a sequence (batch_size=1) with the prime_id
    current_seq = np.full((1, sequence_length), pad_value)
    current_seq[-1][-1] = prime_id
    predicted = [int_to_vocab[prime_id]]
    
    for _ in range(predict_len):
#         if train_on_gpu:
#             current_seq = torch.LongTensor(current_seq).cuda()
#         else:
        current_seq = torch.LongTensor(current_seq)
        
        # initialize the hidden state
        hidden = rnn.init_hidden(current_seq.size(0))
        hidden = [hid.to('cpu') for hid in hidden]
       
        # get the output of the rnn
        output, _ = rnn(current_seq, hidden)
        
        # get the next word probabilities
        p = F.softmax(output, dim=1).data
        if(train_on_gpu):
            p = p.cpu() # move to cpu
         
        # use top_k sampling to get the index of the next word
        top_k = 5
        p, top_i = p.topk(top_k)
        top_i = top_i.numpy().squeeze()
        
        # select the likely next word index with some element of randomness
        p = p.numpy().squeeze()
        word_i = np.random.choice(top_i, p=p/p.sum())
        
        # retrieve that word from the dictionary
        word = int_to_vocab[word_i]
        predicted.append(word)     
        
        if(train_on_gpu):
            current_seq = current_seq.cpu() # move to cpu
        # the generated word becomes the next "current sequence" and the cycle can continue
        if train_on_gpu:
            current_seq = current_seq.cpu()
        current_seq = np.roll(current_seq, -1, 1)
        current_seq[-1][-1] = word_i
    
    gen_sentences = ' '.join(predicted)
    
    # Replace punctuation tokens
    for key, token in token_dict.items():
        ending = ' ' if key in ['\n', '(', '"'] else ''
        gen_sentences = gen_sentences.replace(' ' + token.lower(), key)
    gen_sentences = gen_sentences.replace('\n ', '\n')
    gen_sentences = gen_sentences.replace('( ', '(')
    
    # return all the sentences
    return gen_sentences

### Generate a New Script
It's time to generate the text. Set `gen_length` to the length of TV script you want to generate and set `prime_word` to one of the following to start the prediction:
- "jerry"
- "elaine"
- "george"
- "kramer"

You can set the prime word to _any word_ in our dictionary, but it's best to start with a name for generating a TV script. (You can also start with any other names you find in the original text file!)

In [70]:
# run the cell multiple times to get different results!
gen_length = 400 # modify the length to your preference
prime_word = 'newman' # name for starting the script

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
pad_word = helper.SPECIAL_WORDS['PADDING']
generated_script = generate(model, vocab_to_int[prime_word + ':'], int_to_vocab, token_dict, vocab_to_int[pad_word], gen_length)
print(generated_script)

newman: brody, i think we can get him out of here.

jerry:(reading) gary fogel saw her dude cry.

kramer: oh, you know what, let's go.

jerry: oh, i can't believe i'm gonna get away from the subway!

jerry: i don't think so.

kramer: yeah, well.

jerry: oh, no. no, i'm afraid i can't help him.

jerry: well, you should see a doctor?

kramer: yeah, i guess.

elaine: i can't believe it, i can't believe this.

jerry: you think she'd hit the friendship.

jerry: well, what are you gonna do?

george: i don't know. you can't stand around.

elaine: oh.

jerry:(confused) aww.(holds his nose up)

kramer: well it's not a date, it was the only issue you've ever seen for yourself, huh?

jerry: no, no, i'm afraid he's gonna call you.

elaine:(sighs) well, i think it's fantastic.

jerry: oh, come on, george.

jerry: yeah, i know.

george: well you didn't mention to me that i would possibly care enough for that kind of crap.

george: well, you know what this means, but it's only used to be an actress.


#### Save your favorite scripts

Once you have a script that you like (or find interesting), save it to a text file!

In [71]:
# save script to a text file
f =  open("generated_script_1.txt","w")
f.write(generated_script)
f.close()

# The TV Script is Not Perfect
It's ok if the TV script doesn't make perfect sense. It should look like alternating lines of dialogue, here is one such example of a few generated lines.

### Example generated script

>jerry: what about me?
>
>jerry: i don't have to wait.
>
>kramer:(to the sales table)
>
>elaine:(to jerry) hey, look at this, i'm a good doctor.
>
>newman:(to elaine) you think i have no idea of this...
>
>elaine: oh, you better take the phone, and he was a little nervous.
>
>kramer:(to the phone) hey, hey, jerry, i don't want to be a little bit.(to kramer and jerry) you can't.
>
>jerry: oh, yeah. i don't even know, i know.
>
>jerry:(to the phone) oh, i know.
>
>kramer:(laughing) you know...(to jerry) you don't know.

You can see that there are multiple characters that say (somewhat) complete sentences, but it doesn't have to be perfect! It takes quite a while to get good results, and often, you'll have to use a smaller vocabulary (and discard uncommon words), or get more data.  The Seinfeld dataset is about 3.4 MB, which is big enough for our purposes; for script generation you'll want more than 1 MB of text, generally. 

# Submitting This Project
When submitting this project, make sure to run all the cells before saving the notebook. Save the notebook file as "dlnd_tv_script_generation.ipynb" and save another copy as an HTML file by clicking "File" -> "Download as.."->"html". Include the "helper.py" and "problem_unittests.py" files in your submission. Once you download these files, compress them into one zip file for submission.